In [7]:
from lofar_lst import *
from lofar_planner import *
import tkinter as tk

os.chdir('E:\Dropbox\SETI\Turbo-SETI_Enviroment\Observation_Planning')

In [8]:
# --- Observation Window UTC --- 

start = '2021-08-24 09:00'; end = '2021-08-24 13:00'

UTC_alpha = 1
UTC_beta = 2

station_timezone(end, UTC_alpha)

LST_window(start, end)


 LST Window for UTC 09:00 to 13:00 on the 2021-08-24 is:

 I-LOFAR: 6:39-10:40
LOFAR-SE: 7:59-11:59

 LST difference of ~80 minutes between stations


In [3]:
def dataframe_formatting(fraction_of_ot): 

    """
    Inputs: 
    fraction_of_ot - filters out the percentage of observing time for that falls under a given requirement e.g. 0.5 = 50%

    Outputs: 
    A csv. with possible targets during the given observation window dictated by the parameters of the 'setup' function. 

    """

    combined_targets_dataframe = []
    # --- Organising Dataframes --- 
    combined_targets_dataframe = pd.merge(Station_Alpha[0], Station_Beta[0], on = "target name") # -- merges on title column 
    combined_targets_dataframe = combined_targets_dataframe.rename(columns={"ever observable_x": "Ever Observable(%s)" % observatory_1.name, "ever observable_y": "Ever Observable(%s)" % observatory_2.name, "always observable_x" : "Always Observable(%s)" % observatory_1.name, "always observable_y" : "Always Observable(%s)" % observatory_2.name, "fraction of time observable_x":"Fraction of observable time (%s)" % observatory_1.name, "fraction of time observable_y":"Fraction of observable time (%s)" % observatory_2.name, "target name": "Name"}) # -- renaming cols. 
    combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Ever Observable(%s)' % observatory_1.name] == True]; combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Ever Observable(%s)' % observatory_2.name] == True] # -- removing targets that can't be observed during the window. 
    combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Fraction of observable time (%s)' % observatory_1.name] > fraction_of_ot];  combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Fraction of observable time (%s)' % observatory_2.name] > fraction_of_ot] 
    combined_targets_dataframe = pd.merge(combined_targets_dataframe, Station_Beta[3], on = "Name") # - merges on target name. 
    combined_targets_dataframe.sort_values(by = 'RA', ascending = True, inplace = True) # - sorting values by ascending right ascension. 

    combined_targets_dataframe = combined_targets_dataframe[['Name', 'RA', 'DEC', 'Ever Observable(%s)' % observatory_1.name, 'Always Observable(%s)' % observatory_1.name, 'Ever Observable(%s)' % observatory_2.name, 'Always Observable(%s)' % observatory_2.name, 'Fraction of observable time (%s)' % observatory_1.name , 'Fraction of observable time (%s)' % observatory_2.name]]

    print('Number of possible targets:', len(combined_targets_dataframe))
    # --- Saving .csv ---
    combined_targets_dataframe.to_csv('data/target-lists/combined/Combined-Targets-%s.csv' % (Station_Alpha[2][2:8]), header = True, index  = False)

    return(combined_targets_dataframe)

In [4]:
# %%capture

Station_Alpha = []
Station_Beta = []

observatory_1 = ILOFAR
observatory_2 = ONSULA
catalog = 'data/Simultaneous_SETI_TESS_targets_170621.csv'

# --- Computing Dataframes --- 
Station_Alpha = setup(station_timezone(start, UTC_alpha), station_timezone(end, UTC_alpha), 30, 90, 20, 0.016, observatory_1, catalog, True)
Station_Beta  = setup(station_timezone(start, UTC_beta), station_timezone(end, UTC_beta), 30, 90, 20,  0.016, observatory_2, catalog, False)

test = dataframe_formatting(0.5)

['2021-08-24 10:00:00.000' '2021-08-24 14:00:00.000']
Number of objects that do not meet the criteria of both stations: 273
Number of previously observed targets: 24
Number of objects avalible for observation: 545
['2021-08-24 11:00:00.000' '2021-08-24 15:00:00.000']
Number of objects that do not meet the criteria of both stations: 273
Number of previously observed targets: 24
Number of objects avalible for observation: 545
Number of possible targets: 265


In [5]:
step = 3
target_list = []

for i in range(0, 12):
    t = min(test['RA'], key=lambda x:abs(x - step))
    target_list.append(test.loc[test['RA'] == t]['Name']) 
    test = test.drop(test.loc[test['RA'] == t].index)
    step += 0.25


In [6]:
test = 'TIC142090065.rawspec.0000'

test.split('.')[0]

'TIC142090065'